# Introduction

The goal of this project is to visualise, how the impact of Players born NOT in USA has changed over the years. 

This project uses a kaggle datasets: https://www.kaggle.com/drgilermo/nba-players-stats. We can find 3 .csv files there. Seasons_stats.csv contains stats from every NBA player each season since 1949/1950. Two other files contain some basic informations about each player such as: name, birthdate, height, etc.
The list of international Players comes from Wikipedia: https://en.wikipedia.org/wiki/List_of_foreign_NBA_players

Important note: There is a lot of data that is missing, because not every stat is available since the first season. More details and explaination of each stat: https://www.basketball-reference.com/about/glossary.html

### Imports
Import libraries.

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Visualizations
import plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(theme='white')

# Analysis/Modeling
### Import statistics

In [2]:
stat = pd.read_csv('Seasons_Stats.csv')

### Basic info

In [3]:
stat.shape
stat.dtypes

(24691, 53)

Unnamed: 0      int64
Year          float64
Player         object
Pos            object
Age           float64
Tm             object
G             float64
GS            float64
MP            float64
PER           float64
TS%           float64
3PAr          float64
FTr           float64
ORB%          float64
DRB%          float64
               ...   
2PA           float64
2P%           float64
eFG%          float64
FT            float64
FTA           float64
FT%           float64
ORB           float64
DRB           float64
TRB           float64
AST           float64
STL           float64
BLK           float64
TOV           float64
PF            float64
PTS           float64
Length: 53, dtype: object

### Cleaning

In [4]:
stat = stat.rename(columns={'Unnamed: 0': 'Rk'})
stat.isna().sum()

Rk           0
Year        67
Player      67
Pos         67
Age         75
Tm          67
G           67
GS        6458
MP         553
PER        590
TS%        153
3PAr      5852
FTr        166
ORB%      3899
DRB%      3899
          ... 
2PA         67
2P%        195
eFG%       166
FT          67
FTA         67
FT%        925
ORB       3894
DRB       3894
TRB        379
AST         67
STL       3894
BLK       3894
TOV       5046
PF          67
PTS         67
Length: 53, dtype: int64

Dropping rows where Player is NaN

TODO: Do I have to explain it?

In [5]:
stat = stat.dropna(subset=['Player'])

Dropping two columns with no values

<span style="color:red">Question: Do I have to explain, how do I know that or should I proove it?</span>

In [6]:
stat = stat.drop(['blanl', 'blank2'], axis=1)

# delete asteriks from Hall of Famers
stat['Player'] = stat['Player'].str.replace('*', '')

## Import the list of all time international players

Source: https://en.wikipedia.org/wiki/List_of_foreign_NBA_players

In [7]:
intp = pd.read_csv('international.csv')

In [8]:
intp.shape
intp.dtypes

(698, 8)

Nationality[A]      object
Birthplace[B]       object
Player              object
Pos.                object
Career[C]           object
Yrs                float64
Notes               object
Ref.                object
dtype: object

In [9]:
intp.sample(10)

,Nationality[A],Birthplace[B],Player,Pos.,Career[C],Yrs,Notes,Ref.
174,NaN,(now Croatia),NaN,NaN,NaN,NaN,NaN,NaN
453,Panama,—,Rubén Garcés,F,2000–2001,1.0,—,[460]
123,Canada,United States,Robert Sacre,C,2012–2016,4.0,Born in the United States to an American fathe...,[148]
378,Lithuania,Soviet Union,Martynas Andriuškevičius,C,2005-2007,1.0,"Born in the Soviet Union,[F] represents Lithua...",[393]
429,Nigeria,United States,Al-Farouq Aminu*,F,2010–present,9.0,"Born in the United States to Nigerian parents,...",[432]
572,Slovenia,SFR Yugoslavia,Primož Brezec,C,2001–2008;,8.0,"Born in SFR Yugoslavia,[D] represents Slovenia...",[554]
29,Australia,—,Shane Heal,G,1996–1997;,2.0,—,[41]
156,Croatia,SFR Yugoslavia,Damir Markota,C,2006–2007,1.0,"Born in SFR Yugoslavia,[D] grew up in Sweden, ...",[179]
7,Argentina,—,Nicolás Laprovíttola,G,2016,1.0,—,[17]
386,Lithuania,Soviet Union,Mindaugas Kuzminskas,F,2016–2017,2.0,"Born in the Soviet Union,[F] represents Lithua...",[397]


### Cleaning

Rename columns. Drop 2 of them and all Player NaNs

In [10]:
intp.columns = ['Nationality', 'Birthplace', 'Player', 'Pos', 'Career', 'Years', 'Notes', 'Ref']
intp = intp.drop(['Notes', 'Ref'], axis=1)
intp = intp.dropna(subset=['Player', 'Nationality'])

Drop Players with US Nationality or born in USA


In [11]:
intp = intp.loc[intp.Birthplace != ' United States ']
intp = intp.loc[intp.Nationality != ' United States ']

In [12]:
intp.shape
intp.index = pd.RangeIndex(len(intp))
intp.sample(10)

(420, 6)

,Nationality,Birthplace,Player,Pos,Career,Years
126,Croatia,SFR Yugoslavia,Bruno Šundov,C,1998–2005,7.0
295,Panama,—,Gary Forbes,F,2010–2012,2.0
87,Canada,—,Todd MacCulloch,C,1999–2003,4.0
388,Sweden,—,Jonas Jerebko*,F,2009–present,10.0
134,Czech Republic,Czechoslovakia,Tomáš Satoranský*,F,2016–present,3.0
152,Estonia,Soviet Union,Martin Müürsepp,F,1996–1998,2.0
253,Latvia,Soviet Union,Gundars Vētra,F,1992–1993,1.0
157,France,—,Alexis Ajinça,F,2008–2011; 2013–2018,7.0
3,Argentina,—,Manu Ginóbili,G,2002–2018,16.0
316,Russia,Soviet Union,Sergei Monia,F,2005–2006,1.0


### Check which Names match those from stats data frame

In [13]:
intp.loc[intp.Player.isin(stat.Player)]

,Nationality,Birthplace,Player,Pos,Career,Years
156,France,—,Tariq Abdul-Wahad,F,1997–2003,6.0


Only one player matches. Apparently on the list from Wikipedia there is a trailing space and some other additional signs. Let's get rid of them

In [14]:
intp['Player'] = intp['Player'].str.replace('[*^]', '')
# intp['Player'] = intp['Player'].str.replace('^', '')
intp['Player'] = np.where(intp.Player.str.endswith(' '), intp.Player.str[:-1], intp.Player)

### Decode international letters

In [15]:
from unidecode import unidecode

intp['Player'] = intp['Player'].apply(lambda x: unidecode(x))

In [16]:
intp.Player.isin(stat.Player).value_counts()


True     355
False     65
Name: Player, dtype: int64

Checking, which players still don't match to those from stats and played more than two seasons

In [17]:
# For myself: Never forget parenthesses when using multiple conditions
intp.loc[(~intp.Player.isin(stat.Player)) & (intp.Years > 2.0)]

,Nationality,Birthplace,Player,Pos,Career,Years
35,Austria,—,Jakob Poltl,C,2016–present,3.0
41,Belgium,Zaire,Didier Mbenga,C,2004–2011,7.0
56,Brazil,—,Nene,F/C,2002–present,17.0
66,Cameroon,—,Luc Mbah a Moute,F,2008–present,11.0
123,Croatia,SFR Yugoslavia,Dino Rada,F/C,1993–1997,4.0
201,Germany,West Germany,Christian Welp,C,1987–1990,3.0
301,Puerto Rico,—,Jose Juan Barea,G,2006–present,13.0
313,Russia,Soviet Union,Victor Khryapa,F,2004–2008,4.0
364,Slovenia,SFR Yugoslavia,Radoslav Nesterovic,C,1998–2010,12.0
409,Ukraine,Soviet Union,Slava Medvedenko,F,2000–2007,7.0


Adding those manually

<span style="color:red">Question: Can I do something like this or is it just bad practice? </span>

In [18]:
intp.loc[intp.Player == 'Nene', 'Player'] = 'Nene Hilario'
intp.loc[intp.Player == 'Radoslav Nesterovic', 'Player'] = 'Rasho Nesterovic'
intp.loc[intp.Player == 'Jose Juan Barea', 'Player'] = 'J.J. Barea'
intp.loc[intp.Player == 'Didier Mbenga', 'Player'] = 'Didier Ilunga-Mbenga'
intp.loc[intp.Player == 'Slava Medvedenko', 'Player'] = 'Stanislav Medvedenko'
intp.loc[intp.Player == 'Jakob Poltl', 'Player'] = 'Jakob Poeltl'
intp.loc[intp.Player == 'Victor Khryapa', 'Player'] = 'Viktor Khryapa'
intp.loc[intp.Player == 'Dino Rada', 'Player'] = 'Dino Radja'
intp.loc[intp.Player == 'Luc Mbah a Moute', 'Player'] = 'Luc Mbah'
intp.loc[intp.Player == 'Christian Welp', 'Player'] = 'Chris Welp'

intp.Player.isin(stat.Player).value_counts()
# intp.loc[~intp.Player.isin(stat.Player)].sample(20)

True     365
False     55
Name: Player, dtype: int64

In [19]:
stati = stat.loc[stat.Player.isin(intp.Player)]
stati.shape
stat_by_year = stat.groupby(stat.Year).sum()
stati_by_year = stati.groupby(stati.Year).sum()

(2017, 51)

In [20]:
stati_by_year['PTS'].iplot(kind='line', xTitle='Year', yTitle='Points total')

In [21]:
trace1 = go.Scatter(
    x=stat_by_year.index,
    y=stat_by_year.PTS,
    name='Totals'
)
trace2 = go.Scatter(
    x=stati_by_year.index,
    y=stati_by_year.PTS,
    name='Non-US'
)

data = [trace1, trace2]
layout = go.Layout(
    title=go.layout.Title(
        text='Total points per year',
        xref='paper',
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='Year',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#5f5f5f'
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='Total Points',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#5f5f5f'
            )
        )
    )
)
fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)

In [22]:
grouped = stati.groupby(['Year', 'Player']).count()
traded = grouped.loc[grouped.G > 1]
traded

Rk  Pos  Age  Tm  G  GS  MP  PER  TS%  3PAr  FTr  \
Year   Player                                                                
1954.0 Bob Houbregs        3    3    3   3  3   0   3    3    3     0    3   
1956.0 Chris Harris        3    3    3   3  3   0   3    3    3     0    3   
1979.0 Butch Lee           3    3    3   3  3   0   3    3    3     0    3   
1980.0 Butch Lee           3    3    3   3  3   0   3    3    3     3    3   
1986.0 James Donaldson     3    3    3   3  3   3   3    3    3     3    3   
       Ron Crevier         3    3    3   3  3   3   3    3    3     3    3   
1987.0 Mychal Thompson     3    3    3   3  3   3   3    3    3     3    3   
1989.0 Detlef Schrempf     3    3    3   3  3   3   3    3    3     3    3   
1990.0 Chris Welp          3    3    3   3  3   3   3    3    3     3    3   
       Uwe Blab            3    3    3   3  3   3   3    3    3     3    3   
1991.0 Drazen Petrovic     3    3    3   3  3   3   3    3    3     3    3   
       Mike Smrek          3    3    3   3  3   3   3    3    3     3    3   
       Olden Polynice      3    3    3   3  3   3   3    3    3     3    3   
1992.0 James Donaldson     3    3    3   3  3   3   3    3    3     3    3   
1993.0 Alaa Abdelnaby      3    3    3   3  3   3   3    3    3     3    3   
...                       ..  ...  ...  .. ..  ..  ..  ...  ...   ...  ...   
2016.0 Anderson Varejao    3    3    3   3  3   3   3    3    3     3    3   
       Beno Udrih          3    3    3   3  3   3   3    3    3     3    3   
       Ersan Ilyasova      3    3    3   3  3   3   3    3    3     3    3   
2017.0 Andrew Bogut        3    3    3   3  3   3   3    3    2     2    2   
       Andrew Nicholson    3    3    3   3  3   3   3    3    3     3    3   
       Axel Toupane        3    3    3   3  3   3   3    3    3     3    3   
       Bojan Bogdanovic    3    3    3   3  3   3   3    3    3     3    3   
       Buddy Hield         3    3    3   3  3   3   3    3    3     3    3   
       Ersan Ilyasova      4    4    4   4  4   4   4    4    4     4    4   
       Joffrey Lauvergne   3    3    3   3  3   3   3    3    3     3    3   
       Jose Calderon       3    3    3   3  3   3   3    3    3     3    3   
       Jusuf Nurkic        3    3    3   3  3   3   3    3    3     3    3   
       Omri Casspi         4    4    4   4  4   4   4    4    4     4    4   
       Serge Ibaka         3    3    3   3  3   3   3    3    3     3    3   
       Tyler Ennis         3    3    3   3  3   3   3    3    3     3    3   

                          ORB%  DRB%  TRB%  AST%  STL%  BLK%  TOV%  USG%  OWS  \
Year   Player                                                                   
1954.0 Bob Houbregs          0     0     0     0     0     0     0     0    3   
1956.0 Chris Harris          0     0     0     0     0     0     0     0    3   
1979.0 Butch Lee             3     3     3     3     3     3     3     3    3   
1980.0 Butch Lee             3     3     3     3     3     3     3     3    3   
1986.0 James Donaldson       3     3     3     3     3     3     3     3    3   
       Ron Crevier           3     3     3     3     3     3     3     3    3   
1987.0 Mychal Thompson       3     3     3     3     3     3     3     3    3   
1989.0 Detlef Schrempf       3     3     3     3     3     3     3     3    3   
1990.0 Chris Welp            3     3     3     3     3     3     3     3    3   
       Uwe Blab              3     3     3     3     3     3     3     3    3   
1991.0 Drazen Petrovic       3     3     3     3     3     3     3     3    3   
       Mike Smrek            3     3     3     3     3     3     3     3    3   
       Olden Polynice        3     3     3     3     3     3     3     3    3   
1992.0 James Donaldson       3     3     3     3     3     3     3     3    3   
1993.0 Alaa Abdelnaby        3     3     3     3     3     3     3     3    3   
...                        ...   ...   ...   ...   ...   ...   ...   ...  

In [23]:
traded2 = stat.loc[stat.Tm == 'TOT']
# traded2
stati.isin(traded2)

,Rk,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,...,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
294,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
480,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
612,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
771,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
827,True,True,True,True,True,True,True,False,True,True,True,False,True,False,False,False,False,False,False,False,False,True,True,True,True,...,False,False,False,True,True,True,False,False,False,True,True,True,True,True,True,True,False,False,True,True,False,False,False,True,True
828,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
829,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
900,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
954,True,True,True,True,True,True,True,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,True,True,False,False,False,True,True,True,True,True,True,True,False,False,True,True,False,False,False,True,True
1057,True,True,True,True,True,True,True,False,True,True,True,False,True,False,False,False,False,False,False,False,False,True,True,True,True,...,False,False,False,True,True,True,False,False,False,True,True,True,True,True,True,True,False,False,True,True,False,False,False,True,True


### Import stats from 2018

In [33]:
stats_2018 = pd.read_csv('ignore/stats_2018.csv')
adv_2018 = pd.read_csv('ignore/adv_2018.csv')

### Find rows without stats and delete them

In [34]:
stats_rows_to_delete = stats_2018.loc[stats_2018['Player'] == 'Player']
adv_rows_to_delete = adv_2018.loc[adv_2018['Player'] == 'Player']

stats_2018 = stats_2018.drop(stats_rows_to_delete.index)
adv_2018 = adv_2018.drop(adv_rows_to_delete.index)

Delete columns containing 'Unnamed'

In [35]:
adv_2018 = adv_2018.loc[:, ~adv_2018.columns.str.contains('Unnamed')]

In [36]:
cols = adv_2018.columns.difference(stats_2018.columns)
cols

Index(['3PAr', 'AST%', 'BLK%', 'BPM', 'DBPM', 'DRB%', 'DWS', 'FTr', 'OBPM',
       'ORB%', 'OWS', 'PER', 'STL%', 'TOV%', 'TRB%', 'TS%', 'USG%', 'VORP',
       'WS', 'WS/48'],
      dtype='object')

In [37]:
stats_2018 = stats_2018.join(adv_2019[cols])
stats_2018.shape


(664, 50)

#### A very handy function to find differences in column names

In [38]:
stat.columns.difference(stats_2018.columns)

Index(['Year'], dtype='object')

In [39]:
stats_2018['Year'] = 2018
stats_2018.tail()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,...,BLK,TOV,PF,PTS,3PAr,AST%,BLK%,BPM,DBPM,DRB%,DWS,FTr,OBPM,ORB%,OWS,PER,STL%,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48,Year
685,537,Tyler Zeller,C,28,BRK,42,33,703,125,229,.546,10,26,.385,115,203,.567,.568,40,60,.667,63,131,194,28,...,21,35,78,300,.171,21.9,3.0,6.4,3.4,31.9,4.7,.168,3.0,9.4,5.4,25.5,1.6,9.9,20.5,.573,28.0,5.3,10.1,.193,2018
686,537,Tyler Zeller,C,28,MIL,24,1,406,62,105,.590,0,2,.000,62,103,.602,.590,17,19,.895,47,64,111,19,...,14,12,48,141,.272,26.5,1.8,-0.5,0.0,12.2,2.3,.243,-0.4,3.9,0.1,15.8,1.5,13.5,8.0,.510,27.9,0.7,2.4,.061,2018
687,538,Paul Zipser,SF,23,CHI,54,12,824,81,234,.346,37,110,.336,44,124,.355,.425,19,25,.760,13,118,131,46,...,15,43,86,218,.450,7.4,2.4,-5.1,-1.1,15.4,0.5,.310,-4.0,4.0,-0.3,10.1,1.1,16.7,9.8,.533,21.7,-0.3,0.2,.020,2018
688,539,Ante Žižić,C,21,CLE,32,2,214,49,67,.731,0,0,NaN,49,67,.731,.731,21,29,.724,24,36,60,5,...,13,11,30,119,.605,16.4,0.7,-1.9,-1.6,10.2,1.1,.092,-0.3,0.7,0.1,11.4,1.2,11.4,5.4,.530,21.1,0.0,1.2,.052,2018
689,540,Ivica Zubac,C,20,LAL,43,0,410,61,122,.500,0,1,.000,61,121,.504,.500,39,51,.765,45,78,123,25,...,15,26,47,161,.434,29.4,1.0,3.3,-1.7,11.9,1.9,.267,5.1,1.9,5.5,21.7,1.7,10.1,6.8,.558,31.5,3.9,7.4,.123,2018


### Import stats from 2019

TODO: Source

In [24]:
stats_2019 = pd.read_csv('ignore/stats_2019.csv')
adv_2019 = pd.read_csv('ignore/adv_2019.csv')

### Find rows without stats and delete them

In [25]:
stats_rows_to_delete = stats_2019.loc[stats_2019['Player'] == 'Player']
adv_rows_to_delete = adv_2019.loc[adv_2019['Player'] == 'Player']

stats_2019 = stats_2019.drop(stats_rows_to_delete.index)
adv_2019 = adv_2019.drop(adv_rows_to_delete.index)

Delete columns containing 'Unnamed'

In [26]:
adv_2019 = adv_2019.loc[:, ~adv_2019.columns.str.contains('Unnamed')]

In [27]:
cols = adv_2019.columns.difference(stats_2019.columns)
cols

Index(['3PAr', 'AST%', 'BLK%', 'BPM', 'DBPM', 'DRB%', 'DWS', 'FTr', 'OBPM',
       'ORB%', 'OWS', 'PER', 'STL%', 'TOV%', 'TRB%', 'TS%', 'USG%', 'VORP',
       'WS', 'WS/48'],
      dtype='object')

In [28]:
stats_2019 = stats_2019.join(adv_2019[cols])
stats_2019.shape

(708, 50)

#### A very handy function to find differences in column names

In [29]:
stat.columns.difference(stats_2019.columns)

Index(['Year'], dtype='object')

In [30]:
stats_2019['Year'] = 2019
stats_2019.tail()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,...,BLK,TOV,PF,PTS,3PAr,AST%,BLK%,BPM,DBPM,DRB%,DWS,FTr,OBPM,ORB%,OWS,PER,STL%,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48,Year
729,528,Tyler Zeller,C,29,MEM,4,1,20.5,4.0,7.0,.571,0.0,0.0,NaN,4.0,7.0,.571,.571,3.5,4.5,.778,2.3,2.3,4.5,0.8,...,0.8,1.0,4.0,11.5,.000,6.2,3.5,-1.5,-1.3,12.5,0.1,.643,-0.3,12.1,0.2,19.4,0.6,10.0,12.3,.640,21.8,0.0,0.3,.191,2019
730,529,Ante Zizic,C,22,CLE,59,25,18.3,3.1,5.6,.553,0.0,0.0,NaN,3.1,5.6,.553,.553,1.6,2.2,.705,1.8,3.6,5.4,0.9,...,0.4,1.0,1.9,7.8,.000,7.7,1.8,-3.1,-1.4,22.8,0.3,.399,-1.7,10.6,1.7,16.2,0.6,13.6,16.4,.590,18.2,-0.3,2.0,.087,2019
731,530,Ivica Zubac,C,21,TOT,59,37,17.6,3.6,6.4,.559,0.0,0.0,NaN,3.6,6.4,.559,.559,1.7,2.1,.802,1.9,4.2,6.1,1.1,...,0.9,1.2,2.3,8.9,.000,9.1,4.0,-0.3,1.0,24.2,1.3,.332,-1.2,11.9,1.9,18.9,0.6,13.9,18.2,.604,20.3,0.5,3.2,.148,2019
732,530,Ivica Zubac,C,21,LAL,33,12,15.6,3.4,5.8,.580,0.0,0.0,NaN,3.4,5.8,.580,.580,1.7,2.0,.864,1.6,3.3,4.9,0.8,...,0.8,1.0,2.2,8.5,.000,7.3,4.3,-0.9,0.4,21.2,0.6,.342,-1.3,11.0,1.1,19.5,0.4,12.9,16.2,.633,20.5,0.1,1.8,.165,2019
733,530,Ivica Zubac,C,21,LAC,26,25,20.2,3.8,7.2,.538,0.0,0.0,NaN,3.8,7.2,.538,.538,1.7,2.3,.733,2.3,5.3,7.7,1.5,...,0.9,1.4,2.5,9.4,.000,10.9,3.6,0.3,1.5,27.2,0.7,.323,-1.2,12.7,0.7,18.3,0.9,14.8,20.2,.574,20.1,0.3,1.4,.132,2019


In [31]:
stat.columns.difference(stats_2019.columns)

Index([], dtype='object')

In [32]:
stat.shape
stat.tail()

(24624, 51)

,Rk,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,...,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
24686,24686,2017.0,Cody Zeller,PF,24.0,CHO,62.0,58.0,1725.0,16.7,0.604,0.002,0.442,8.6,17.3,12.9,9.1,1.8,3.0,10.9,15.5,3.4,2.2,5.6,0.157,...,2.3,2.1,1.8,253.0,443.0,0.571,0.0,1.0,0.000,253.0,442.0,0.572,0.571,133.0,196.0,0.679,135.0,270.0,405.0,99.0,62.0,58.0,65.0,189.0,639.0
24687,24687,2017.0,Tyler Zeller,C,27.0,BOS,51.0,5.0,525.0,13.0,0.508,0.006,0.247,9.2,17.0,13.2,12.2,0.7,3.3,10.2,16.5,0.5,0.6,1.0,0.094,...,0.8,-2.5,-0.1,78.0,158.0,0.494,0.0,1.0,0.000,78.0,157.0,0.497,0.494,22.0,39.0,0.564,43.0,81.0,124.0,42.0,7.0,21.0,20.0,61.0,178.0
24688,24688,2017.0,Stephen Zimmerman,C,20.0,ORL,19.0,0.0,108.0,7.3,0.346,0.000,0.161,10.8,24.9,17.6,5.3,0.9,3.7,8.3,14.8,-0.1,0.1,0.0,-0.005,...,0.4,-7.3,-0.1,10.0,31.0,0.323,0.0,0.0,NaN,10.0,31.0,0.323,0.323,3.0,5.0,0.600,11.0,24.0,35.0,4.0,2.0,5.0,3.0,17.0,23.0
24689,24689,2017.0,Paul Zipser,SF,22.0,CHI,44.0,18.0,843.0,6.9,0.503,0.448,0.181,1.9,14.2,8.0,6.1,0.9,1.5,14.4,14.4,-0.3,0.8,0.5,0.030,...,-0.1,-3.7,-0.4,88.0,221.0,0.398,33.0,99.0,0.333,55.0,122.0,0.451,0.473,31.0,40.0,0.775,15.0,110.0,125.0,36.0,15.0,16.0,40.0,78.0,240.0
24690,24690,2017.0,Ivica Zubac,C,19.0,LAL,38.0,11.0,609.0,17.0,0.547,0.013,0.206,7.1,21.9,14.3,8.1,1.1,4.4,10.4,20.3,0.6,0.5,1.1,0.086,...,0.3,-2.5,-0.1,126.0,238.0,0.529,0.0,3.0,0.000,126.0,235.0,0.536,0.529,32.0,49.0,0.653,41.0,118.0,159.0,30.0,14.0,33.0,30.0,66.0,284.0


### Add stats from last two years

In [40]:
stat = pd.concat([stat, stats_2018], sort=False)
stat = stat.reset_index(drop=True)

stat = pd.concat([stat, stats_2019], sort=False)
stat = stat.reset_index(drop=True)

In [41]:
stat.shape
stat.tail()

(25996, 51)

,Rk,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,...,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
25991,528,2019.0,Tyler Zeller,C,29,MEM,4,1,20.5,19.4,.640,.000,.643,12.1,12.5,12.3,6.2,0.6,3.5,10.0,21.8,0.2,0.1,0.3,.191,...,-1.3,-1.5,0.0,4.0,7.0,.571,0.0,0.0,NaN,4.0,7.0,.571,.571,3.5,4.5,.778,2.3,2.3,4.5,0.8,0.3,0.8,1.0,4.0,11.5
25992,529,2019.0,Ante Zizic,C,22,CLE,59,25,18.3,16.2,.590,.000,.399,10.6,22.8,16.4,7.7,0.6,1.8,13.6,18.2,1.7,0.3,2.0,.087,...,-1.4,-3.1,-0.3,3.1,5.6,.553,0.0,0.0,NaN,3.1,5.6,.553,.553,1.6,2.2,.705,1.8,3.6,5.4,0.9,0.2,0.4,1.0,1.9,7.8
25993,530,2019.0,Ivica Zubac,C,21,TOT,59,37,17.6,18.9,.604,.000,.332,11.9,24.2,18.2,9.1,0.6,4.0,13.9,20.3,1.9,1.3,3.2,.148,...,1.0,-0.3,0.5,3.6,6.4,.559,0.0,0.0,NaN,3.6,6.4,.559,.559,1.7,2.1,.802,1.9,4.2,6.1,1.1,0.2,0.9,1.2,2.3,8.9
25994,530,2019.0,Ivica Zubac,C,21,LAL,33,12,15.6,19.5,.633,.000,.342,11.0,21.2,16.2,7.3,0.4,4.3,12.9,20.5,1.1,0.6,1.8,.165,...,0.4,-0.9,0.1,3.4,5.8,.580,0.0,0.0,NaN,3.4,5.8,.580,.580,1.7,2.0,.864,1.6,3.3,4.9,0.8,0.1,0.8,1.0,2.2,8.5
25995,530,2019.0,Ivica Zubac,C,21,LAC,26,25,20.2,18.3,.574,.000,.323,12.7,27.2,20.2,10.9,0.9,3.6,14.8,20.1,0.7,0.7,1.4,.132,...,1.5,0.3,0.3,3.8,7.2,.538,0.0,0.0,NaN,3.8,7.2,.538,.538,1.7,2.3,.733,2.3,5.3,7.7,1.5,0.4,0.9,1.4,2.5,9.4


# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here